In [ ]:
import numpy as np
import pygetm

# Note: "%matplotlib widget" below enables interactive plots but requires https://github.com/matplotlib/ipympl
# Alternatively you could use "%matplotlib notebook" (deprecated).
%matplotlib widget
import matplotlib.pyplot

In [ ]:
# Set up rectangular domain
domain = pygetm.domain.create_cartesian(
    np.linspace(0.0, 50000, 100),
    np.linspace(0.0, 30000, 30),
    periodic_x=True,
    lat=0.0,
    H=50,
)
sim = pygetm.Simulation(domain, runtype=pygetm.RunType.BAROTROPIC_2D)

# Idealized surface forcing
tausx = sim.U.array(fill=0.01)
tausy = sim.V.array(fill=0.0)
sp = sim.T.array(fill=0.0)
fwf = sim.T.array(fill=0.0)

# Time
timestep = 10.0
ntime = int(3600.0 // timestep)

# Set up velocity plot
nsample = 4
fig, ax = matplotlib.pyplot.subplots()
pc = ax.pcolormesh(sim.X.x, sim.X.y, sim.T.z, vmin=-0.001, vmax=0.001)
cb = fig.colorbar(pc)
cb.set_label("elevation (m)")
u = sim.momentum.U.interp(sim.T) / sim.T.D
v = sim.momentum.V.interp(sim.T) / sim.T.D
Q = ax.quiver(
    sim.T.x[::nsample, ::nsample],
    sim.T.y[::nsample, ::nsample],
    u[::nsample, ::nsample],
    v[::nsample, ::nsample],
    scale=0.005,
)
title = ax.set_title("time: 0 s")

In [ ]:
plotting_interval = 1
times = timestep * np.arange(ntime)
Us = np.empty_like(times)
for istep, time in enumerate(times):
    Us[istep] = np.mean(sim.momentum.U)

    sim.update_surface_pressure_gradient(sim.T.z, sp)
    sim.momentum.advance_depth_integrated(timestep, tausx, tausy, sim.dpdx, sim.dpdy)
    sim.advance_surface_elevation(timestep, sim.momentum.U, sim.momentum.V, fwf)
    sim.update_depth()

    # Update velocity plot
    if istep % plotting_interval == 0:
        u = sim.momentum.U.interp(sim.T) / sim.T.D
        v = sim.momentum.V.interp(sim.T) / sim.T.D
        Q.set_UVC(u[::nsample, ::nsample], v[::nsample, ::nsample])
        title.set_text("time: %s s" % time)
        pc.set_array(sim.T.z[...].ravel())
        fig.canvas.draw()

In [ ]:
fig, ax = matplotlib.pyplot.subplots()
ax.plot(times, Us)
ax.set_title("mean U")
ax.grid()

# Let's try this with an island in the center of the domain

In [ ]:
# Set up rectangular domain with and circular island in the center
domain = pygetm.domain.create_cartesian(
    np.linspace(0.0, 50000, 100),
    np.linspace(0.0, 30000, 30),
    periodic_x=True,
    lat=0.0,
    H=50,
)
distance_from_center = np.sqrt((domain.x - 25000) ** 2 + (domain.y - 15000) ** 2)
domain.mask[distance_from_center < 5000] = 0
sim = pygetm.Simulation(domain, runtype=pygetm.RunType.BAROTROPIC_2D)

# Idealized surface forcing
tausx = sim.U.array(fill=0.01)
tausy = sim.V.array(fill=0.0)
sp = sim.T.array(fill=0.0)
fwf = sim.T.array(fill=0.0)

# Time
timestep = 5.0
ntime = int(3600.0 // timestep)

# Set up velocity plot
nsample = 1
fig, ax = matplotlib.pyplot.subplots()
masked_z = sim.T.z.ma
pc = ax.pcolormesh(sim.X.x, sim.X.y, masked_z, vmin=-0.001, vmax=0.001)
cb = fig.colorbar(pc)
cb.set_label("elevation (m)")
u, v = sim.momentum.U.interp(sim.T) / sim.T.D, sim.momentum.V.interp(sim.T) / sim.T.D
Q = ax.quiver(
    sim.T.x[::nsample, ::nsample],
    sim.T.y[::nsample, ::nsample],
    u.ma[::nsample, ::nsample],
    v.ma[::nsample, ::nsample],
    scale=0.01,
)
title = ax.set_title("time: 0 s")
V = sim.momentum.V.ma[:-1, :]

In [ ]:
plotting_interval = 5
times = timestep * np.arange(ntime)
Us = np.empty_like(times)
for istep, time in enumerate(times):
    Us[istep] = np.mean(sim.momentum.U)

    sim.update_surface_pressure_gradient(sim.T.z, sp)
    sim.momentum.advance_depth_integrated(timestep, tausx, tausy, sim.dpdx, sim.dpdy)
    sim.advance_surface_elevation(timestep, sim.momentum.U, sim.momentum.V, fwf)
    sim.update_depth()

    V_asym = (V + V[::-1, :]).max() - (V + V[::-1, :]).min()
    V_range = V.max() - V.min()
    print("Asymmetry in V: %s" % (V_asym / V_range if V_asym > 0 else 0.0,))

    # Update velocity plot
    if istep % plotting_interval == 0:
        u, v = (
            sim.momentum.U.interp(sim.T) / sim.T.D,
            sim.momentum.V.interp(sim.T) / sim.T.D,
        )
        Q.set_UVC(u.ma[::nsample, ::nsample], v.ma[::nsample, ::nsample])
        title.set_text("time: %s s" % time)
        pc.set_array(masked_z.ravel())
        fig.canvas.draw()